In [1]:
%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
from pathlib import Path

In [2]:
KeyNotInclude = ['PhraseTextBlock', 'ScratchPad', 'Sleep', 'NextSuggestions', 'SpaceBar', 'NextPhrase', 'Quit', 'YesQuestionResult', 'IncreaseDwellTime', 'DecreaseDwellTime']

In [3]:
def CharNotActivated(userKeys):
    nKey = 0
    LetterLookedAt = [userKeys[letterInd] for letterInd in range(0,len(userKeys))  if userKeys[letterInd][1] not in KeyNotInclude and 'Suggestion' not in userKeys[letterInd][1]]
    LetterLookingAt = list()
                
    for keyInd in range(0, len(LetterLookedAt)):
        if keyInd == 0:
            theLetter = LetterLookedAt[keyInd][1]
            LetterLookingAt.append(LetterLookedAt[keyInd])
            continue
                        
        checkLetter = LetterLookedAt[keyInd][1]
                    
        if theLetter != checkLetter:
            theLetter = checkLetter
                        
            LetterLookedAtList.append(LetterLookingAt)
            LetterLookingAt = list()
            LetterLookingAt.append(LetterLookedAt[keyInd])
            
        else:
            LetterLookingAt.append(LetterLookedAt[keyInd])
    
    for key in LetterLookedAtList:
        for progressInd in key:
            if float(progressInd[2]) == 1:
                LetterLookedAtList.remove(LetterLookedAtList[nKey])
                #continue
                break
        nKey = nKey + 1
    
    return len(LetterLookedAtList)

In [4]:
def CharTyped(scratchPad):
    phraseUser = list()
    phraseUserEnd = list()
                
    for row_ind in range(0, len(scratchPad)):
        if row_ind!= 0 and scratchPad[row_ind][1] == '':
            phraseUser.append(scratchPad[row_ind-1])
                        
    for row_ind2 in range(0, len(phraseUser)):
        if len(phraseUser[row_ind2])>2:
            phraseUser[row_ind2][1] = ''.join(phraseUser[row_ind2][1:])
        phraseUserEnd.append(phraseUser[row_ind2][1])
              
    return len(''.join(phraseUserEnd))

In [7]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
    

In [9]:
metricComputed = 'attended_but_not_selected_rate'
AttendedButNotSelectedRateList = list()
Names = list()

subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    userKeys = None
    
    if not dirs:
        
        if 'notCompleted' in root:
            continue
        if 'tb' in root or 'joha' in root:
            continue

        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            else:
                continue
            
            if userKeys and scratchPad:
                
                nCharNotActivated = CharNotActivated(userKeys)                
                nCharTyped = CharTyped(scratchPad)
                
                nAttendedButNotSelected = float(nCharNotActivated/nCharTyped)
                
                AttendedButNotSelectedRateList.append(nAttendedButNotSelected)
                
                # save only the name of the subject and the date part of the file
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                print(a.findall(root))
                Names.append(a.findall(root)[0])
            
if AttendedButNotSelectedRateList:
    AddDataToFile(metricComputed, Names, AttendedButNotSelectedRateList)

['akt_MS']
['be_DT']
['jl_DT']
['KEA_MS']
['lone_DT']
['mcc_MS']
['MK_DT']
['MT_MS']
['ok_MS']
['pt_DT']
['sc_MS']
['sh_MS']
['slh_DT']
['ae_DT']
['ep_DT']
['hc_MS']
['ib_MS']
['js_MS']
['km_DT']
['ma_DT']
['mw_MS']
['pgba_DT']
['pt_MS']
['smn_DT']
['eo_DT']
['jek_MS']
['jg_DT']
['lg_MS']
['lr_MS']
['mm_MS']
['mr_DT']
['snk_DT']
C:\DTU\Data\201805_HealthnRehab\data_summary.csv
adding the metric
timestamp
adding the metric
5/15/2018 14:03
adding the metric
5/15/2018 14:56
adding the metric
5/15/2018 10:21
adding the metric
5/15/2018 12:35
adding the metric
5/15/2018 11:11
adding the metric
5/15/2018 12:12
adding the metric
5/15/2018 14:44
adding the metric
5/15/2018 10:58
adding the metric
5/15/2018 12:28
adding the metric
5/15/2018 12:51
adding the metric
5/15/2018 14:48
adding the metric
5/15/2018 15:42
adding the metric
5/15/2018 11:48
adding the metric
5/16/2018 15:13
adding the metric
5/16/2018 13:12
adding the metric
5/16/2018 13:55
adding the metric
5/16/2018 10:51
adding the met

In [ ]:
len(LetterLookedAtList)